## Cargamos las librerias

In [16]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import ast


## Cargamos el archivo json y pasamos los datos a un dataframe

In [2]:
ruta_reviews = "data2/australian_user_reviews.json"
filas_review = []
with open(ruta_reviews, encoding='utf-8') as f:
    for line in f.readlines():
        filas_review.append(ast.literal_eval(line))

# Transformamos a un dataframe
df_reviews = pd.DataFrame(filas_review)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [3]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [4]:
df_reviews.isnull().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

Podemos verificar que todos nuestros datos son de tipo objeto y que no tenemos ninguno que sea nulo

Verificamos si hay filas duplicadas en funcion del user_id y si son validas

In [5]:
duplicados = df_reviews[df_reviews.duplicated("user_id", keep=False)]
duplicados

,user_id,user_url,reviews
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,"[{'funny': '', 'posted': 'Posted June 16.', 'l..."
50,Rivtex,http://steamcommunity.com/id/Rivtex,"[{'funny': '', 'posted': 'Posted December 23, ..."
83,76561198094224872,http://steamcommunity.com/profiles/76561198094...,[]
119,DieMadchenschanderin,http://steamcommunity.com/id/DieMadchenschanderin,"[{'funny': '', 'posted': 'Posted August 29, 20..."
147,relesprit,http://steamcommunity.com/id/relesprit,"[{'funny': '', 'posted': 'Posted December 27, ..."
...,...,...,...
17819,76561198076474887,http://steamcommunity.com/profiles/76561198076...,"[{'funny': '', 'posted': 'Posted April 12.', '..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
18028,76561198075591109,http://steamcommunity.com/profiles/76561198075...,"[{'funny': '', 'posted': 'Posted December 26, ..."
18234,76561198092022514,http://steamcommunity.com/profiles/76561198092...,"[{'funny': '', 'posted': 'Posted July 3.', 'la..."


Podemos observar que tenemos 623 registros duplicados en cuanto a user_id pero puede ser que un mismo usuario haya realizado mas de un comentario por lo que se procede a analizar

In [13]:
user_id = "76561198156664158"
user_rows = duplicados.loc[duplicados["user_id"] == user_id, "reviews"]
for review_list in user_rows:
    for review in review_list:
        print(review)
    print("-" * 40)

{'funny': '', 'posted': 'Posted June 16.', 'last_edited': '', 'item_id': '252950', 'helpful': '0 of 1 people (0%) found this review helpful', 'recommend': True, 'review': 'love it'}
----------------------------------------
{'funny': '', 'posted': 'Posted June 16.', 'last_edited': '', 'item_id': '252950', 'helpful': '0 of 1 people (0%) found this review helpful', 'recommend': True, 'review': 'love it'}
----------------------------------------


Aqui se tomo un usuario para verificar si era el mismo comentario repetido y estabamos en lo correcto por lo que se procede a eliminar

In [15]:
df_reviews = df_reviews.drop_duplicates(subset="user_id", keep=False)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


En la variable reviews se observo que muchos de los registros se encuentran de forma anidada se analiza la estructura


In [14]:
df_reviews["reviews"][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

Corresponde desanidar la columna para luego poder proceder con el analisis. En este caso procedemos con la funcion "explode" para poder convertir cada elemento de la lista en una fila separada, replicando el índice y las demás columnas según sea necesario.

In [21]:
df_reviews_expanded = df_reviews.explode("reviews")
df_reviews_normalized = json_normalize(df_reviews_expanded["reviews"])
df_reviews_expanded.reset_index(drop=True, inplace=True)
df_reviews_normalized.reset_index(drop=True, inplace=True)
df_reviews_final = pd.concat([df_reviews_expanded,df_reviews_normalized], axis=1)
df_reviews_final = df_reviews_final.drop(columns='reviews')
df_reviews_final.head(2)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.


In [22]:
df_reviews_final.shape

(57595, 9)

In [23]:
df_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57595 entries, 0 to 57594
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      57595 non-null  object
 1   user_url     57595 non-null  object
 2   funny        57568 non-null  object
 3   posted       57568 non-null  object
 4   last_edited  57568 non-null  object
 5   item_id      57568 non-null  object
 6   helpful      57568 non-null  object
 7   recommend    57568 non-null  object
 8   review       57568 non-null  object
dtypes: object(9)
memory usage: 4.0+ MB


En el  caso de las nuevas columnas generadas observamos que no tenemos valores nulos pero que si hay registros que no tienen informacion por lo que se verifica

In [24]:
conteo_valores_vacios = df_reviews_final.apply(lambda col: col[col == ""].count())
conteo_valores_vacios

user_id            0
user_url           0
funny          49698
posted             0
last_edited    51632
item_id            0
helpful            0
recommend          0
review            30
dtype: int64

Solo tres columnas contienen valores vacios en el caso de la columna "funny" y "last_edited" no son necesarias para nuestro analisis por lo que se procede a eliminarlas y en cuanto a "reviews" se concluye que varios usuarios no dieron un comentario

In [25]:
df_reviews_final = df_reviews_final.drop(columns=["funny","last_edited"])
df_reviews_final

,user_id,user_url,posted,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
57590,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 10.,70,No ratings yet,True,a must have classic from steam definitely wort...
57591,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 8.,362890,No ratings yet,True,this game is a perfect remake of the original ...
57592,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
57593,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 20.,730,No ratings yet,True,:D
